In [1]:
import pandas as pd
import numpy as np

In [288]:
# display the plot in the notebook 
# same as %matplotlib inline
from bokeh.plotting import output_notebook
output_notebook() 

Loading BokehJS ...

# Basic Static Plot

In [328]:
nr_models = 10
short_controls = ['age', 'grade', 'iq_score', 'female']

df1 = pd.DataFrame()

for i in range(nr_models):
    ctrl_names = short_controls
    factor = 0.5
    nr_params = len(ctrl_names)
    df = pd.DataFrame()
    df['param_name'] = ctrl_names
    df['param_value'] = factor * np.arange(nr_params) + np.random.normal(0, 1, nr_params).round(1)
    df['model'] = 'model_' + str(i)
    
    df['conf_int_lower'] = df['param_value'] - np.random.normal(1, factor, nr_params).round(1)
    df['conf_int_upper'] = df['param_value'] + np.random.normal(1, factor, nr_params).round(1)

    
    df1 = pd.concat([df1, df], axis=0, sort=False)
    
df1.reset_index(inplace=True, drop=True)
df1[['model', 'param_name', 'param_value']].head()

,model,param_name,param_value
0,model_0,age,-1.7
1,model_0,grade,0.4
2,model_0,iq_score,0.3
3,model_0,female,0.9
4,model_1,age,-1.2


In [329]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show

def static_comparison_plot(df):
    source = ColumnDataSource(df)
    
    # create the "canvas"
    plot = figure(
        title="Comparison Plot",
        y_range=df["param_name"].unique())
    
    # add circles representing the parameter value
    circle_glyph = plot.circle(
        source=source,
        x="param_value",
        y="param_name",
        size=12)
    
    show(plot)

In [330]:
static_comparison_plot(df1)

This already supports some basic interactivity out of the box:
- zooming
- moving

# Add Whiskers

In [336]:
from bokeh.models import Arrow

def comparison_plot_with_whiskers(df):
    source = ColumnDataSource(df)
    
    # create the "canvas"
    plot = figure(
        title="Comparison Plot",
        y_range=df["param_name"].unique())
    
    # add circles representing the parameter value
    circle_glyph = plot.circle(
        source=source,
        x="param_value",
        y="param_name",
        size=12)
    
    # =============================================================
    plot.add_layout(
        Arrow(
            source=source, 
            # y_range_name="param_name", 
            x_start="conf_int_lower", 
            x_end="conf_int_upper",)
    )
    # =============================================================

    show(plot)

In [337]:
comparison_plot_with_whiskers(df1)

# Display Additional Information when Hovering

In [130]:
from bokeh.models import HoverTool


def comparison_plot_with_hovering(df):
    source = ColumnDataSource(df)
    
    # create the "canvas"
    plot = figure(
        title="Comparison Plot",
        y_range=df["param_name"].unique())
    
    # add circles representing the parameter value
    circle_glyph = plot.circle(
        source=source,
        x="param_value",
        y="param_name",
        size=12)
    
    # =============================================================
    # HoverTool 
    tooltips = [
        ("parameter value", "@param_value"),
        ("model", "@model")]
    hover = HoverTool(renderers=[circle_glyph], tooltips=tooltips)
    plot.tools.append(hover)
    # =============================================================

    show(plot)

In [131]:
comparison_plot_with_hovering(df1)

# Select points of the same model when one estimate is selected

In [47]:
# two components of Bokeh
# need custom callback when point is tapped
# server OR JavaScript
# ...

In [235]:
%%javascript

function select_model_points(source){

// adapted from https://stackoverflow.com/a/44996422
var chosen = source.selected.indices;
if (typeof(chosen) == "number"){
    var chosen = [chosen]
};

var chosen_models = []; 

for (var i = 0; i < chosen.length; ++ i){
    chosen_models.push(source.data['model'][chosen[i]])
};
        
var chosen_models_indices = []; 

for (var i = 0; i < source.data['index'].length; ++ i){
    if (chosen_models.includes(source.data['model'][i])){
        chosen_models_indices.push(i)
    };
};

source.selected.indices = chosen_models_indices 
source.change.emit();
    
}

<IPython.core.display.Javascript object>

In [239]:
with open('callback.js', 'r') as f:
    js_code = f.read()

In [245]:
from bokeh.models import CustomJS

def comparison_plot_with_model_selection(df, js_code=js_code):
    # safety measure to make sure the index 
    # gives the position in the arrays 
    # that the source data dictionary points to
    safe_df = df.reset_index(drop=True)
    
    source = ColumnDataSource(safe_df)
    
    # create the "canvas"
    plot = figure(
        title="Comparison Plot",
        y_range=df["param_name"].unique())
    
    # add circles representing the parameter value
    circle_glyph = plot.circle(
        source=source,
        x="param_value",
        y="param_name",
        size=12)
    
    # HoverTool
    tooltips = [
        ("parameter value", "@param_value"),
        ("model", "@model")]
    hover = HoverTool(renderers=[circle_glyph], tooltips=tooltips)
    plot.tools.append(hover)
    
    # =============================================================
    # TapTool
    js_kwargs = {"source": source}
    js_callback = CustomJS(args=js_kwargs, code=js_code)
    tap = TapTool(callback=js_callback)
    plot.tools.append(tap)
    # =============================================================

    show(plot)

In [246]:
comparison_plot_with_model_selection(df1)

In [316]:
def styled_comparison_plot(df, js_code=js_code):
    # safety measure to make sure the index 
    # gives the position in the arrays 
    # that the source data dictionary points to
    safe_df = df.reset_index(drop=True)
    
    source = ColumnDataSource(safe_df)
    
    # create the "canvas"
    plot = figure(
        title="Comparison Plot",
        y_range=df["param_name"].unique(),
        
        # =============================================================
        plot_width=600,
        plot_height=300, 
        # =============================================================
    )
        
    # add circles representing the parameter value
    circle_glyph = plot.circle(
        source=source,
        x="param_value",
        y="param_name",
        size=12,
        
        # =============================================================
        color='#DAA520', 
        alpha=0.5,
        nonselection_color="#035096",
        selection_color="firebrick",
        nonselection_alpha=0.2,
        selection_alpha=0.7,
        # =============================================================
    )

    # HoverTool
    tooltips = [
        ("parameter value", "@param_value"),
        ("model", "@model")]
    hover = HoverTool(renderers=[circle_glyph], tooltips=tooltips)
    plot.tools.append(hover)
    
    # TapTool
    js_kwargs = {"source": source}
    js_callback = CustomJS(args=js_kwargs, code=js_code)
    tap = TapTool(callback=js_callback)
    plot.tools.append(tap)

    show(plot)

In [317]:
source = styled_comparison_plot(df1)

# Add groups of parameters to be plotted into separate plots

## Create dataset with parameter groups

In [264]:
df2 = pd.DataFrame()

nr_models = 10
nr_intercepts = 5

intercept_tuples = [str(i) for i in range(nr_intercepts)]
short_controls = ['age', 'grade', 'iq_score', 'gender']
long_controls = short_controls + [
    'parent_educ', 'parent_inc', 'parent_occup', 'dist_to_school', 
    'single_parent', 'single_parent_x_parent_inc']

cols = ['model', 'param_value', 'param_name', 'conf_int_lower', 'conf_int_upper', 'group']

for i in range(nr_models):
    if i < 0.7 * nr_models:
        ctrl_names = short_controls
        factor = 0.5
    else:
        ctrl_names = long_controls
        factor = 0.7
    nr_params = nr_intercepts + len(ctrl_names)
    
    df = pd.DataFrame(columns=cols)
    df['param_name'] = [str(i) for i in range(nr_intercepts)] + ctrl_names
    df['param_value'] = factor * np.arange(nr_params) + np.random.normal(0, 1, nr_params)
    df['conf_int_lower'] = df['param_value'] - np.random.normal(1, factor, nr_params)
    df['conf_int_upper'] = df['param_value'] + np.random.normal(1, factor, nr_params)
    df['group'] = ['intercept'] * nr_intercepts + [r'$\beta$'] * len(ctrl_names)
    df['model'] = 'model_' + str(i)
    
    df2 = pd.concat([df2, df], axis=0, sort=False)
    
df2.reset_index(inplace=True, drop=True)
df2 = df2[['model', 'group', 'param_name', 'conf_int_lower', 'param_value', 'conf_int_upper']]
df2[:9]

,model,group,param_name,conf_int_lower,param_value,conf_int_upper
0,model_0,intercept,0,1.645099,2.748241,3.239338
1,model_0,intercept,1,-0.314079,1.313745,2.300364
2,model_0,intercept,2,-1.431843,1.501342,3.382452
3,model_0,intercept,3,0.666232,2.975476,4.760225
4,model_0,intercept,4,1.720036,2.126216,2.624553
5,model_0,$\beta$,age,-0.284375,0.661715,0.991299
6,model_0,$\beta$,grade,2.365945,2.849575,2.848893
7,model_0,$\beta$,iq_score,4.294427,4.701923,6.987490
8,model_0,$\beta$,gender,4.172048,5.432398,5.839550


In [320]:
from bokeh.layouts import gridplot

def grouped_comparison_plot(df, js_code=js_code):
    # safety measure to make sure the index 
    # gives the position in the arrays 
    # that the source data dictionary points to
    safe_df = df.reset_index(drop=True)
    
    source = ColumnDataSource(safe_df)
    
    plots = []
    for group_name in df['group'].unique():
    
        # create the "canvas"
        group_plot = figure(
            title="Comparison Plot of {} Parameters".format(group_name.title()),
            y_range=df[df['group'] == group_name]["param_name"].unique(),
            plot_width=600,
            plot_height=300, 
        )

        # add circles representing the parameter value
        circle_glyph = group_plot.circle(
            source=source,
            x="param_value",
            y="param_name",
            size=12,
            color='#035096', 
            alpha=0.5,
            nonselection_color="#035096",
            selection_color="firebrick",
            nonselection_alpha=0.2,
            selection_alpha=0.7,
        )
    
        # HoverTool
        tooltips = [
            ("parameter value", "@param_value"),
            ("model", "@model")]
        hover = HoverTool(renderers=[circle_glyph], tooltips=tooltips)
        group_plot.tools.append(hover)

        # TapTool
        js_kwargs = {"source": source}
        js_callback = CustomJS(args=js_kwargs, code=js_code)
        tap = TapTool(callback=js_callback)
        group_plot.tools.append(tap)
        
        plots.append(group_plot)

    grid = gridplot(plots, toolbar_location="right", ncols=1)
    show(grid)

In [321]:
grouped_comparison_plot(df2)

# Add confidence intervals as whiskers that are only displayed when the model is selected

# Add interactive legend

In [295]:
l = [[1], [2], [3, 4]]

In [296]:
[x for sub_l in l for x in sub_l]

[1, 2, 3, 4]